In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import itertools
import statsmodels.api as sm
import kds
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn import metrics
from matplotlib import rcParams
from termcolor import colored as cl
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from xgboost.sklearn import XGBClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
# Create a dataframe with imported csv file
df = pd.read_csv(r'C:\Users\kirka\OneDrive\Documents\Coursework\Data Prep\week 5\creditcard.csv')

In [2]:
X = df.drop(['Class','Time','Amount','V6','V8','V13','V15','V23'], axis=1)
y = df['Class']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

In [4]:
df_train, df_test = train_test_split(df, test_size = 0.3, stratify=df['Class'], random_state=888)
features = df_train.drop(columns=['Class','Time','Amount','V6','V8','V13','V15','V23']).columns

In [5]:
df_test_y = df_test['Class']
df_test_y

43446     0
170593    0
223374    0
197224    0
41941     0
         ..
198999    0
247149    0
147383    0
175286    0
154478    0
Name: Class, Length: 85443, dtype: int64

In [6]:
msk_negative = df_train['Class'] == 0
msk_positive = df_train['Class'] == 1

In [7]:
df_negative_undersample = df_train[msk_negative].sample(n=msk_positive.sum(), random_state = 888)
df_train_undersample = pd.concat([df_negative_undersample, df_train[msk_positive]])

In [8]:
df_train_undersample['Class'].value_counts()

0    344
1    344
Name: Class, dtype: int64

In [9]:
clf = LogisticRegression(random_state=888)

In [10]:
clf.fit(df_train_undersample[features], df_train_undersample['Class'])
y_pred = clf.predict_proba(df_test[features])[:,1]

In [11]:
clf.fit(df_train[features], df_train['Class'])

LogisticRegression(random_state=888)

In [12]:
y_pred = clf.predict_proba(df_test[features])[:,1]

In [13]:
roc_auc_score(df_test['Class'], y_pred)

0.963951421378586

In [14]:
kds.metrics.decile_table(df_test_y, y_pred)

LABELS INFO:

 prob_min         : Minimum probability in a particular decile
 prob_max         : Minimum probability in a particular decile
 prob_avg         : Average probability in a particular decile
 cnt_events       : Count of events in a particular decile
 cnt_resp         : Count of responders in a particular decile
 cnt_non_resp     : Count of non-responders in a particular decile
 cnt_resp_rndm    : Count of responders if events assigned randomly in a particular decile
 cnt_resp_wiz     : Count of best possible responders in a particular decile
 resp_rate        : Response Rate in a particular decile [(cnt_resp/cnt_cust)*100]
 cum_events       : Cumulative sum of events decile-wise 
 cum_resp         : Cumulative sum of responders decile-wise 
 cum_resp_wiz     : Cumulative sum of best possible responders decile-wise 
 cum_non_resp     : Cumulative sum of non-responders decile-wise 
 cum_events_pct   : Cumulative sum of percentages of events decile-wise 
 cum_resp_pct     : Cu

,decile,prob_min,prob_max,prob_avg,cnt_cust,cnt_resp,cnt_non_resp,cnt_resp_rndm,cnt_resp_wiz,resp_rate,cum_cust,cum_resp,cum_resp_wiz,cum_non_resp,cum_cust_pct,cum_resp_pct,cum_resp_pct_wiz,cum_non_resp_pct,KS,lift
0,1,0.001,1.000,0.014,8545.0,134.0,8411.0,14.8,148,1.568,8545.0,134.0,148,8411.0,10.001,90.541,100.0,9.861,80.680,9.053
1,2,0.001,0.001,0.001,8544.0,4.0,8540.0,14.8,0,0.047,17089.0,138.0,148,16951.0,20.000,93.243,100.0,19.873,73.370,4.662
2,3,0.000,0.001,0.000,8544.0,2.0,8542.0,14.8,0,0.023,25633.0,140.0,148,25493.0,30.000,94.595,100.0,29.888,64.707,3.153
3,4,0.000,0.000,0.000,8545.0,2.0,8543.0,14.8,0,0.023,34178.0,142.0,148,34036.0,40.001,95.946,100.0,39.904,56.042,2.399
4,5,0.000,0.000,0.000,8544.0,3.0,8541.0,14.8,0,0.035,42722.0,145.0,148,42577.0,50.001,97.973,100.0,49.917,48.056,1.959
5,6,0.000,0.000,0.000,8544.0,1.0,8543.0,14.8,0,0.012,51266.0,146.0,148,51120.0,60.000,98.649,100.0,59.933,38.716,1.644
6,7,0.000,0.000,0.000,8545.0,2.0,8543.0,14.8,0,0.023,59811.0,148.0,148,59663.0,70.001,100.000,100.0,69.949,30.051,1.429
7,8,0.000,0.000,0.000,8544.0,0.0,8544.0,14.8,0,0.000,68355.0,148.0,148,68207.0,80.001,100.000,100.0,79.966,20.034,1.250
8,9,0.000,0.000,0.000,8544.0,0.0,8544.0,14.8,0,0.000,76899.0,148.0,148,76751.0,90.000,100.000,100.0,89.983,10.017,1.111
9,10,0.000,0.000,0.000,8544.0,0.0,8544.0,14.8,0,0.000,85443.0,148.0,148,85295.0,100.000,100.000,100.0,100.000,0.000,1.000


In [15]:
combine_data = pd.DataFrame({"Actual": df_test_y, "Predictions": y_pred})

In [16]:
combine_data['score_rank'] = pd.qcut(combine_data['Predictions'],100,labels=['r1','r2','r3','r4','r5','r6','r7',
    'r8','r9','r10','r11','r12','r13','r14','r15','r16','r17','r18','r19','r20','r21', 'r22', 'r23','r24','r25',
'r26','r27','r28','r29','r30','r31','r32','r33','r34','r35','r36','r37','r38','r39','r40','r41','r42','r43','r44','r45','r46',
'r47','r48','r49','r50','r51','r52','r53','r54','r55','r56','r57','r58','r59','r60','r61','r62','r63','r64','r65','r66','r67',
'r68','r69','r70','r71','r72','r73','r74','r75','r76','r77','r78','r79','r80','r81','r82','r83','r84','r85','r86','r87','r88',
'r89','r90','r91','r92','r93','r94','r95','r96','r97','r98','r99','r100'])

In [17]:
combine_data

,Actual,Predictions,score_rank
43446,0,0.000148,r41
170593,0,0.000086,r26
223374,0,0.002144,r98
197224,0,0.000629,r85
41941,0,0.000401,r73
...,...,...,...
198999,0,0.000455,r77
247149,0,0.000229,r55
147383,0,0.000087,r26
175286,0,0.001676,r96


In [18]:
pd.crosstab(combine_data['score_rank'],combine_data['Actual'])

Actual,0,1
score_rank,,
r1,855,0
r2,854,0
r3,855,0
r4,854,0
r5,855,0
...,...,...
r96,854,1
r97,855,0
r98,852,2


In [19]:
lr_model = sm.Logit(df_train_undersample['Class'], df_train_undersample[features])

In [20]:
result = lr_model.fit(method = 'newton')

Optimization terminated successfully.
         Current function value: 0.276690
         Iterations 15


In [21]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Class   No. Observations:                  688
Model:                          Logit   Df Residuals:                      665
Method:                           MLE   Df Model:                           22
Date:                Sat, 13 Aug 2022   Pseudo R-squ.:                  0.6008
Time:                        18:11:30   Log-Likelihood:                -190.36
converged:                       True   LL-Null:                       -476.89
Covariance Type:            nonrobust   LLR p-value:                3.909e-107
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
V1            -2.0041      0.224     -8.927      0.000      -2.444      -1.564
V2             1.7803      0.240      7.428      0.000       1.311       2.250
V3            -3.8385      0.436     -8.797      0.000      -4.694      -2.983
V4             2.5930      0.289      8.977      0.000       2.027       3.159
V5            -2.3530      0.304     -7.729      0.000      -2.950      -1.756
V7            -5.3236      0.619     -8.595      0.000      -6.538      -4.110
V9            -2.3428      0.304     -7.695      0.000      -2.940      -1.746
V10           -5.6125      0.675     -8.319      0.000      -6.935      -4.290
V11            3.9969      0.484      8.258      0.000       3.048       4.946
V12           -7.0025      0.836     -8.374      0.000      -8.641      -5.364
V14           -7.9449      0.911     -8.719      0.000      -9.731      -6.159
V16           -6.6113      0.793     -8.338      0.000      -8.165      -5.057
V17          -11.9876      1.409     -8.506      0.000     -14.750      -9.225
V18           -4.1525      0.539     -7.710      0.000      -5.208      -3.097
V19            1.5307      0.241      6.359      0.000       1.059       2.002
V20            0.3796      0.207      1.835      0.066      -0.026       0.785
V21            3.4532      0.489      7.058      0.000       2.494       4.412
V22           -0.8665      0.218     -3.973      0.000      -1.294      -0.439
V24            0.2154      0.197      1.094      0.274      -0.171       0.601
V25            0.6861      0.274      2.502      0.012       0.149       1.224
V26           -0.2571      0.299     -0.860      0.390      -0.843       0.329
V27            2.0028      0.598      3.348      0.001       0.830       3.175
V28            0.0521      0.800      0.065      0.948      -1.516       1.620
==============================================================================

Possibly complete quasi-separation: A fraction 0.41 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [22]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.601      
Dependent Variable: Class            AIC:              426.7258   
Date:               2022-08-13 18:11 BIC:              531.0030   
No. Observations:   688              Log-Likelihood:   -190.36    
Df Model:           22               LL-Null:          -476.89    
Df Residuals:       665              LLR p-value:      3.9095e-107
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     15.0000                                       
--------------------------------------------------------------------
          Coef.     Std.Err.      z      P>|z|     [0.025     0.975]
--------------------------------------------------------------------
V1        -2.0041     0.2245   -8.9274   0.0000    -2.4440   -1.5641
V2         1.7803     0.2397    7.4282   0.0000     1.3106    2.2501
V3        -3.8385     0.4363   -8.7968   0.0000    -4.6937   -2.9832
V4         2.5930     0.2889    8.9769   0.0000     2.0269    3.1591
V5        -2.3530     0.3044   -7.7291   0.0000    -2.9497   -1.7563
V7        -5.3236     0.6194   -8.5950   0.0000    -6.5375   -4.1096
V9        -2.3428     0.3045   -7.6948   0.0000    -2.9395   -1.7461
V10       -5.6125     0.6747   -8.3186   0.0000    -6.9349   -4.2901
V11        3.9969     0.4840    8.2579   0.0000     3.0482    4.9455
V12       -7.0025     0.8362   -8.3744   0.0000    -8.6414   -5.3636
V14       -7.9449     0.9112   -8.7192   0.0000    -9.7307   -6.1590
V16       -6.6113     0.7929   -8.3379   0.0000    -8.1654   -5.0572
V17      -11.9876     1.4094   -8.5057   0.0000   -14.7499   -9.2253
V18       -4.1525     0.5386   -7.7100   0.0000    -5.2081   -3.0969
V19        1.5307     0.2407    6.3588   0.0000     1.0589    2.0025
V20        0.3796     0.2068    1.8355   0.0664    -0.0257    0.7850
V21        3.4532     0.4892    7.0583   0.0000     2.4943    4.4121
V22       -0.8665     0.2181   -3.9726   0.0001    -1.2939   -0.4390
V24        0.2154     0.1969    1.0940   0.2740    -0.1705    0.6013
V25        0.6861     0.2742    2.5025   0.0123     0.1488    1.2235
V26       -0.2571     0.2991   -0.8596   0.3900    -0.8433    0.3291
V27        2.0028     0.5982    3.3481   0.0008     0.8304    3.1752
V28        0.0521     0.8002    0.0651   0.9481    -1.5163    1.6204
==================================================================

"""